In [18]:
# Importamos las librerias necesarias.
import pandas as pd
import numpy as np
from scipy import stats

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
# Creamos un DataFrame de nuestros datos.
car_data = pd.read_csv('../data/cars.csv', sep=';')

# Mostramos la forma y los primeros registros de nuestro DataFrame.
print(car_data.shape)
car_data.head()

(58049, 25)


,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,REV_Garantia,Averia_grave,QUEJA_CAC,EDAD_COCHE,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,NO DATA,Averia muy grave,SI,2.08,2892,0,0,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,SI,No,NO,2.45,1376,7187,0,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,NO DATA,No,NO,2.51,1376,0,1,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,SI,Averia muy grave,SI,2.43,2015,7256,1,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,NO DATA,No,NO,2.30,1818,0,1,3,21,3


In [20]:
# Revisamos si hay valores nulos en nuestro DataFrame.
valores_nulos = car_data.isnull().sum()

# Mostramos los valores nulos.
valores_nulos

CODE                   0
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia_              0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona _Renta        13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
EDAD_COCHE             0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad Cliente           0
Tiempo                 0
dtype: int64

In [21]:
# Revisamos si hay valores duplicados en nuestro DataFrame.
duplicados = car_data.duplicated().sum()

# Mostramos los valores duplicados.
duplicados

0

In [22]:
# Revisamos los tipos de datos de nuestro DataFrame.
tipos_de_datos = car_data.dtypes

# Mostramos los tipos de datos.
tipos_de_datos

CODE                object
PRODUCTO            object
TIPO_CARROCERIA     object
COMBUSTIBLE         object
Potencia_           object
TRANS               object
FORMA_PAGO          object
ESTADO_CIVIL        object
GENERO              object
OcupaciOn           object
PROVINCIA           object
Campanna1           object
Campanna2           object
Campanna3           object
Zona _Renta         object
REV_Garantia        object
Averia_grave        object
QUEJA_CAC           object
EDAD_COCHE         float64
COSTE_VENTA          int64
km_anno              int64
Mas_1_coche          int64
Revisiones           int64
Edad Cliente         int64
Tiempo               int64
dtype: object

Identificamos putliers de 2 maneras: IQR y Z-score.

Método IQR: Los valores fuera del rango [𝑄1 − 1.5 × 𝐼𝑄𝑅, 𝑄3 + 1.5 × 𝐼𝑄𝑅] se consideran outliers.
Método Z-score: Los valores con un z-score absoluto mayor a 3 se consideran outliers (a más de 3 desviaciones estándar de la media).

In [23]:
# Calcular el IQR para identificar outliers
Q1 = car_data.quantile(0.25)  # Primer cuartil (25%)
Q3 = car_data.quantile(0.75)  # Tercer cuartil (75%)
IQR = Q3 - Q1           # Rango intercuartílico

# Definir los límites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Encontrar los outliers
outliers = (car_data < lower_bound) | (car_data > upper_bound)

# Mostrar las filas que contienen outliers
print(car_data[outliers.any(axis=1)])

           CODE PRODUCTO TIPO_CARROCERIA COMBUSTIBLE Potencia_ TRANS  \
302      CLI303        A           TIPO1      FUEL 1      Baja     M   
1296    CLI1297        A           TIPO1      FUEL 2     Media     M   
1675    CLI1676        B           TIPO1      FUEL 1      Baja     M   
1739    CLI1740        B           TIPO1      FUEL 1     Media     M   
1908    CLI1909        B           TIPO1      FUEL 1     Media     M   
...         ...      ...             ...         ...       ...   ...   
57913  CLI55887        I           TIPO7      FUEL 2     Media     A   
58033  CLI57672        E           TIPO2      FUEL 1     Media     M   
58034  CLI57679        E           TIPO2      FUEL 1     Media     M   
58044  CLI57905        F           TIPO3      FUEL 1      Alta     M   
58047  CLI58038        G           TIPO5      FUEL 2     Media     M   

             FORMA_PAGO ESTADO_CIVIL GENERO OcupaciOn  ... REV_Garantia  \
302               Otros       CASADO      F   Empresa  ...  

C:\Users\Laura Rodríguez\AppData\Local\Temp\ipykernel_6452\2927218837.py:11: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = (car_data < lower_bound) | (car_data > upper_bound)


In [24]:
# Calcular los z-scores
z_scores = stats.zscore(car_data)

# Identificar los outliers (z-score > 3 o z-score < -3)
outliers_z = (abs(z_scores) > 3)

# Mostrar las filas que contienen outliers
print(car_data[outliers_z.any(axis=1)])

TypeError: can only concatenate str (not "float") to str